---
# 신경망 종합 테스트
- burst 하여 active cell 이 많으면 새로운 패턴을 인지한다는 의미
- 
---

In [2]:
import numpy as np
import random
import nbimporter
import Encoder
import SP3
import TM

Importing Jupyter notebook from Encoder.ipynb
Importing Jupyter notebook from SP3.ipynb
Importing Jupyter notebook from TM.ipynb
Importing Jupyter notebook from memoryStruct.ipynb


In [101]:
encoder = Encoder.ScalarEncoder(out_size=100, w=30, max_val=30)
sp = SP3.SpatialPooler(input_size=encoder.out_size, columns=512, minOver=5)
tm = TM.TemporalMemory(sdr_size=sp.columnCount)

In [102]:
for i in range(10):
    a = np.arange(20)
    random.shuffle(a)
    
    for i in a:
        input_data = encoder.encode(i)
        sp.compute_SDR(input_data)

In [91]:
for i in range(30):
    sp.compute_SDR(encoder.encode(i), learn=False)
    sp.viz_activeCol()

Loading BokehJS ...

active columns : [31, 39, 51, 58, 66, 72, 109, 158, 321, 356, 368, 372, 390, 490] 
count : 14


Loading BokehJS ...

active columns : [6, 31, 34, 39, 72, 104, 109, 158, 191, 321, 322, 368, 372] 
count : 13


Loading BokehJS ...

...

active columns : [11, 26, 28, 73, 140, 193, 271, 276, 400, 408] 
count : 10


Loading BokehJS ...

active columns : [11, 28, 73, 119, 140, 193, 235, 342, 408, 429] 
count : 10


Loading BokehJS ...

KeyboardInterrupt: 

In [3]:
for i in range(29):
    sp.compute_SDR(encoder.encode(i), learn=False)
    sdr1 = sp.getActiveColumns()
    sp.compute_SDR(encoder.encode(i+1), learn=False)
    sdr2 = sp.getActiveColumns()

    sp.similarity(sdr1, sdr2)

Loading BokehJS ...

유사도 : 0.6153846153846154
[11, 32, 62, 69, 71, 94, 97, 99, 110, 124, 146, 186, 226, 243, 271, 351, 377, 420, 437, 476, 501, 506]
[11, 62, 94, 97, 99, 110, 113, 124, 146, 186, 243, 271, 276, 288, 377, 420, 437, 471, 476, 501]


Loading BokehJS ...

유사도 : 0.52
[11, 62, 94, 97, 99, 110, 113, 124, 146, 186, 243, 271, 276, 288, 377, 420, 437, 471, 476, 501]
[8, 62, 94, 97, 99, 110, 128, 186, 202, 243, 271, 288, 377, 395, 437, 471, 501, 506]


Loading BokehJS ...

...

유사도 : 0.42105263157894735
[23, 53, 54, 128, 186, 226, 243, 262, 351, 365, 370, 381, 384, 419, 438, 458, 507]
[23, 54, 128, 136, 226, 243, 266, 365, 381, 438]


Loading BokehJS ...

유사도 : 0.5384615384615384
[23, 54, 128, 136, 226, 243, 266, 365, 381, 438]
[23, 54, 128, 186, 226, 243, 262, 381, 420, 438]


Loading BokehJS ...

유사도 : 0.375
[23, 54, 128, 186, 226, 243, 262, 381, 420, 438]
[23, 54, 99, 124, 128, 130, 136, 146, 163, 186, 226, 243, 359, 365, 381, 402, 419, 420, 438, 458, 459, 462, 506]


In [137]:
seq_data = [3]

for i in range(1):
    for s in seq_data:
        dat = encoder.encode(s)
        sp.compute_SDR(dat)
        sdr = sp.getActiveColumns()
        tm.compute(sdr)

In [138]:
# len(tm.connInfo.totalSegments)
print('1918 번 cell 에 연결되어 있는 synapse :', tm.connInfo.totalCells[432].connectedSynapses)
print(len(tm.prev_activeCells))
print(len(tm.activeCells))
tm.prev_activeCells & tm.activeCells

1918 번 cell 에 연결되어 있는 synapse : set()
85
55


{60,
 61,
 62,
 63,
 64,
 325,
 326,
 327,
 328,
 329,
 405,
 406,
 407,
 408,
 409,
 1575,
 1576,
 1577,
 1578,
 1579}